In [1]:
import requests

import pandas as pd
import sqlite3

from tqdm import tqdm

In [2]:
reqRegiones = requests.get("https://api-colombia.com/api/v1/Region").json()


In [3]:
regiones = [{k: v for k, v in ele.items() if k != "departments"} for ele in reqRegiones]
dfRegiones = pd.DataFrame(regiones)

In [4]:
reqDepartaments = [requests.get(f"https://api-colombia.com/api/v1/Region/{region}/departments").json() for region in tqdm(range(1, 7))]

  0%|                                                                                                                                                  | 0/6 [00:00<?, ?it/s]

 17%|███████████████████████                                                                                                                   | 1/6 [00:00<00:03,  1.41it/s]

 33%|██████████████████████████████████████████████                                                                                            | 2/6 [00:01<00:02,  1.75it/s]

 50%|█████████████████████████████████████████████████████████████████████                                                                     | 3/6 [00:01<00:01,  1.92it/s]

 67%|████████████████████████████████████████████████████████████████████████████████████████████                                              | 4/6 [00:02<00:01,  1.92it/s]

 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████                       | 5/6 [00:02<00:00,  1.97it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  2.03it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.93it/s]

In [5]:
dfsDeparments = []

for region in tqdm(reqDepartaments):
    deparment = [{k: v for k, v in ele.items() if v!= None and not isinstance(v, dict)} for ele in region]
    dfsDeparments.append(pd.DataFrame(deparment))

  0%|                                                                                                                                                  | 0/6 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 2464.82it/s]

In [6]:
dfDeparments = pd.concat(dfsDeparments)
total_Region_Dep = dfDeparments['regionId'].nunique()
if total_Region_Dep != 6:
    raise Exception("The total of regions are 6")

In [7]:
conn = sqlite3.connect('applications.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS Departmnets')
cursor.execute('DROP TABLE IF EXISTS Regions')
dfDeparments.to_sql('Departmnets', conn, if_exists='replace', index=False)
dfRegiones.to_sql('Regions', conn, if_exists='replace', index=False)

6